In machine learning and data science, we are often equipped with tons of data. Indeed, given the constellation of packages to query data services, free and open source data sets, and the rapid and persistent collection of geographical data, there is simply too much data to even represent coherently in a single, tidy fashion. However, we often need to be able to construct useful features from this rich and deep sea of data.

機械学習やデータサイエンスでは、多くの場合、大量のデータを扱います。実際、データサービスを照会するためのパッケージ、フリーでオープンソースのデータセット、地理データの迅速かつ持続的な収集などを考えると、単にデータが多すぎて、単一の整然とした方法で表現することさえできない。しかし、この豊かで深いデータの海から有用な機能を構築することは、しばしば必要とされる。

Where data is available, but not yet directly usable, feature engineering helps to construct representations or transformations of the original data that are more useful for modeling a given phenomenon of interest. In traditional machine learning, feature engineering involves applying additional domain knowledge to raw information in order to structure it in a manner that is meaningful for a model. Often, this involves some sort of transformation of the original dataset, which is a well-studied concept in both classical statistics and remains so in machine learning methods. While feature engineering always relies on this implicit domain knowledge, it is an extremely important part of adapting general-purpose algorithms to unique or distinctive problems facing the every-day data scientist.

データが入手可能だが、まだ直接利用できない場合、特徴量工学は、目的の現象をモデル化するためにより有用な表現または変換を元のデータから構築することを支援する。従来の機械学習では、素性工学は、モデルにとって意味のある方法で構造化するために、生の情報に領域知識を追加して適用する。これは古典的な統計学でよく研究されている概念であり、機械学習の手法においても同様である。特徴量エンジニアリングは常にこの暗黙のドメイン知識に依存しているが、日々のデータサイエンティストが直面するユニークで特徴的な問題に汎用アルゴリズムを適応させる上で、非常に重要な部分である。

Geography is one of the most high-quality, ubiquitous ways to introduce domain knowledge into a problem: everything has a position in both space and time. And, while things that happen near to one another in time do not necessarily have a fundamental relationship, things that are near one another are often related. Thus, space is the ultimate linkage key, allowing us to connect different datasets together in order to improve our models and our predictions. This means that, even for aspatial, “non-geographic” data, you can use spatial feature engineering to create useful, highly-relevant features for your analysis.

地理は、問題に領域知識を導入するための最も質の高い、どこにでもある方法の1つである。また、時間的に近くにあるものは必ずしも基本的な関係を持たないが、近くにあるものは関連性があることが多い。このように、空間は究極のリンケージキーであり、モデルや予測を向上させるために、異なるデータセットを結びつけることができるのです。つまり、非空間的な「非地理的」データであっても、空間特徴工学を用いることで、分析に有用で関連性の高い特徴を作成することができるのです。

At its core, spatial feature engineering is the process of developing additional information from raw data using geographic knowledge. This distilling of information can occur between datasets, where geography is used to link information in separate datasets together; or within datasets, where geography can be used to augment the information available for one sample by borrowing from nearby ones. This chapter is structured following that distinction: for cases where geography connects different datasets, we adopt the term “Map Matching”, often used in industry; while we use the mirroring concept of “Map Synthesis” describing the use of geographical structure to derive new features from a given dataset. Technically speaking, some of the methods we review are similar across these two cases, or even the same; however they can be applied in the context of “matching” or “synthesis”, and we consider those conceptually different, hence their inclusion in both sections. Throughout the chapter, we use the AirBnB nightly rental prices in San Diego, as well as auxiliary datasets such as elevation or Census demographics.

Spatial Feature Engineeringとは、地理的な知識を用いて生データから追加情報を抽出するプロセスです。この情報の抽出は、データセット間で行われ、地理を利用して別々のデータセットの情報を結びつける場合と、データセット内で行われ、地理を利用してあるサンプルの情報を近くのデータセットから借りて補強する場合がある。本章は、この区別に従って構成されています。地理が異なるデータセットをつなぐ場合、産業界でよく使われる「マップマッチング」という用語を採用し、一方、与えられたデータセットから新しい特徴を導き出すために地理的構造を使用することを表す「マップシンセシス」というミラーリング概念を使用します。技術的に言えば、私たちがレビューする手法のいくつかは、これら2つのケースで類似しているか、あるいは同じである。しかし、それらは「マッチング」または「合成」の文脈で適用され、概念的に異なると考えられるため、両方のセクションに含めることにした。この章では、サンディエゴのAirBnBのレンタル価格と、標高や国勢調査の人口統計などの補助的なデータセットを使用します。



In [ ]:
import geopandas, pandas, libpysal.weights as weights, contextily
import matplotlib.pyplot as plt
import cenpy
import numpy
import osmnx
import seaborn
import rasterio
from rasterio.plot import show as rioshow

Throughout this chapter, we will use the same dataset to which we want to append more information or augment, and we will do so through leveraging geography. For the illustration, we will use the set of AirBnB properties we have used in the previous chapter. Let’s read it:

この章では、より多くの情報を追加したり、補強したい同じデータセットを使用し、地理を活用することでそれを実現することにします。説明のために、前の章で使用した AirBnB のプロパティのセットを使用します。読んでみましょう。

# What is Spatial Feature Engineering?

At its core, spatial feature engineering is the process of developing additional information from raw data using geographic knowledge. This synthesis could occur between datasets, where geography is used to link samples in separate datasets together; or within datasets, where geography can be used to borrow information from nearby samples. Building linkages between datasets is often called “Map Matching”, while we use the term “Map Synthesis” to describe the use of geographical structure to derive new features from existing data. We will cover both kinds of feature engineering.

spatial feature engineeringとは、地理的な知識を用いてrawデータから追加情報を抽出するプロセスである。この合成は、**データセット間**（別々のデータセットのサンプルを結びつけるために地理を使用する場合）や、**データセット内**（近くのサンプルの情報を借用するために地理を使用する場合）で行われることがあります。
**データセット間のリンクを構築することを「Map Matching」と呼び**ますが、**既存のデータから新しい特徴を導き出すために地理的構造を使用することを「Map Synthesis」**と呼びます。本講演では、この2種類のfeature engineeringを取り上げます。

## 空間要約特徴量(Spatial Summary Feature) & 補間特徴量(Interpolated Feature) / 近接特徴量(Proximity Feature)

To help us discuss this, a vocabulary is helpful. We will cover a few different kinds of features in this chapter, both of which can be constructed in either Map Synthesis or Map Matching operations:

これを議論するために、語彙があると便利である。この章ではいくつかの異なる種類の特徴を取り上げるが、いずれもマップ合成またはマップマッチング操作のいずれかで構築できるものである。

- A **spatial summary feature** measures the attributes of observations that have some pre-specified spatial relationship with our target observations. This includes **空間要約特徴量**は、我々のターゲット観測と何らかの事前に指定された空間的関係を持つ観測の属性を測定します。これには以下が含まれます
  - taking the average or median value of features within a neighborhood of each target observation. **各観測点の近傍にある特徴量の平均値または中央値を取る**こと。
  - the spatial lag, used in previous chapters of this book (e.g., Chapters 3, 6, and 11, is a kind of spatial summary feature, since it reflects the average value of the data in the neighborhood around each point. 本書のこれまでの章（第3章、第6章、第11章など）で使用した空間ラグとは、各点周辺のデータの平均値を反映するため、空間要約機能の一種である。
  - Other kinds of spatial summary features might include the count of observations within a given distance or the standard deviation of ten nearest observations. 他の種類の空間要約機能には、与えられた距離内のオブザベーションのカウントや、10個の最も近いオブザベーションの標準偏差が含まれるかもしれません。
  - Summary features generally include **interpolated features** which involve a transfer of information from one spatial support to another, such as when the target locations are not the same as the locations in our ancillary data. Interpolated features become significantly more complex as well when the data is areal, as will be discussed later in this chapter. 要約特徴量には、一般に、**ある空間サポートから別の空間サポートへの情報の転送を伴う補間特徴量**が含まれ、例えば、ターゲット位置が補助データの位置と同じでない場合などがある。後述するように、データが面的である場合にも補間特徴量は著しく複雑になる。

- A **proximity feature** measures the distance from a target observation to some other observation or position in the map. This might be done in a map matching context, as we did before with the distance to Balboa Park in Chapter 11, or it might be done in a map synthesis context by measuring the distance to the nearest other observation. **近接特徴量**は、**ターゲット観測からマップ上の他の観測または位置までの距離**を測定します。これは、11章のバルボア公園への距離で行ったように、マップマッチングコンテキストで行われるかもしれませんし、マップ合成のコンテキストで、最も近い他の観測点への距離を測定することで行われるかもしれません。

# Feature Engineering Using Map Matching

Space is the ultimate linkage key; map matching is all about exploiting this asset. Geographic information is collected at different scales, aggregated using bespoke geographic delineations, and ultimately stored in different datasets. Modeling and machine learning usually require the use of data aligned and coherently structured; to bring disparate geo-datasets together, the data scientist needs to be able to transfer information expressed for one geography into another. This is where map matching comes to the rescue. By using geographical location, we can connect datasets that have no common key or that are otherwise completely unrelated. This is a “magic trick” that we can pull off because of location that would be much harder, or impossible, with data lacking spatial information.

空間は究極のリンクキーであり、マップマッチングはこの資産を活用することに他ならない。地理情報は様々なスケールで収集され、特注の地理的区分を用いて集約され、最終的に異なるデータセットに保存される。モデリングや機械学習では、通常、整合性のある、首尾一貫した構造のデータを使用する必要があります。異種の地理データセットをまとめるために、データサイエンティストは、ある地理で表現された情報を別の地理に転送できるようにする必要があります。そこで、マップマッチングが役に立ちます。地理的な位置を利用することで、共通のキーがない、あるいは全く関係のないデータセット同士をつなげることができる。これは、空間情報がないデータでは難しい、あるいは不可能な、位置情報があるからこそできる「魔法のトリック」なのです。


The details, and difficulty, of this transfer of information from one geography to another one depend on the nature of the “source” and “target”, and on the precision with which we want to perform such transfer. In this context, there is always an inevitable trade-off between precision and sophistication: more precise transfer is usually possible at the expense of more advanced and involved techniques. Their additional cost in applying them, both in terms of cognitive load on the data scientist’s part or in terms of computation, must be weighted in on an individual basis: sometimes we need the best possible estimates, at all costs; sometimes “good enough” is, well, good enough.

ある地理から別の地理への情報伝達の詳細と難易度は、「ソース」と「ターゲット」の性質と、その伝達をどの程度の精度で行うかによって決まる。この文脈では、正確さと精巧さの間に常にトレードオフが存在する。より正確な情報伝達は通常、より高度で複雑な技術を犠牲にすることで可能となる。**データサイエンティスト側の認識負荷や計算負荷など、技術適用に伴う追加コストは、個別に評価する必要があります**。

In this section, we cover a few cases that we consider represent the most common and widely used approaches. We begin with situations where we have two point datasets and all we need to calculate are descriptive statistics of one dataset to the observations in the other. We switch the type of data and consider how to attach information from a continuous grid, stored in a raster file, to a set of points. Then we show the equivalent case for “moving” information for a polygon geography to a point dataset. These are all cases that, in their simplest form, involve mostly traditional GIS operations (e.g., buffer construction, spatial joins) and little in the way of statistical modeling. As an example of a case that is more involved, we cover the transfer of information from a set of points to another set of points in different locations, and of a polygon geography to another different polygon geography. For these final cases, but also as more generally, we try to include examples that capture the essence of the method, but keep the technique as simple as possible. For almost any of these cases we cover, the reader can find more sophisticated techniques that usually yield more accurate estimates. Where possible, we try to signpost these.

このセクションでは、最も一般的で広く使われているアプローチを表すと思われるいくつかのケースを取り上げます。
- まず、**2つのポイントデータセットがあり、片方のデータセットの記述統計量ともう片方のデータセットのオブザベーションを計算する必要がある場合**から始めます。
- 次に、データの種類を変えて、**ラスターファイルに格納された連続グリッドからの情報を、点の集合にどのように添付するか**を検討します。
- 次に、ポリゴン地形の情報をポイントデータセットに「移動」させる場合の等価性を示す。？？
- 上の3つはすべて、最も単純な形で、ほとんど伝統的なGIS操作（バッファの構築、空間結合など）を伴い、統計的なモデリングはほとんど行わないケースである。
- より複雑なケースの例として、
  - 点の集合から異なる場所にある別の点の集合への情報の転送、ポリゴン地形から別のポリゴン地形への情報の転送を取り上げます。これらの最終的な事例だけでなく、より一般的な事例として、手法の本質を捉え、かつ手法をできるだけシンプルにした事例を掲載するようにしています。これらの事例のほとんどについて、読者はより正確な推定値を得ることができる、より洗練された手法を見つけることができます。可能な限り、そのようなテクニックを紹介するようにします。

## Counting nearby features

## Assigning point values from surfaces: elevation of AirBnBs

## Point Interpolation using scikit-learn

## Polygon to Point

## Area to area interpolation 面積から面積への補間

# Feature Engineering using Map Synthesis

## Spatial Summary Features in Map Synthesis

## Clustering as Feature Engineering